# Preprocessing of scraped recipes

### Import libraries

In [33]:
import pandas as pd
import re
import pickle
import numpy as np
pd.set_option('display.max_colwidth', None)
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')
import en_core_web_sm
stop_words = set([w.lower() for w in list(STOP_WORDS)])
nlp.Defaults.stop_words |= {" f "," s "," etc"}


### Load the data

In [240]:
json_file = 'recipes.pkl'
set_json = pickle.load(open(json_file, 'rb'))
type(set_json)

list

In [241]:
filename='recipes.pkl'
df = pd.DataFrame(pd.read_pickle(filename))
df.head(3)

,Recipe,INSTRUCTIONS
0,"[1 tablespoon ground flaxseed, 3 tablespoons water, 1½ cups all-purpose flour, spooned and leveled, ½ teaspoon baking soda, ¼ teaspoon baking powder, ½ teaspoon sea salt, ½ cup coconut oil, soft but not melted]","Preheat the oven to 350°F and line a baking sheet with parchment paper.\n\nIn a small bowl, combine the flaxseed and water and set aside to thicken for 5 minutes.\n\nIn a medium bowl, combine the (spooned and leveled) flour, baking soda, baking powder, and salt.\n\nIn a large bowl, combine the coconut oil, cane sugar, brown sugar, almond butter, and vanilla. Stir in the flaxseed mixture and mix well.\n\nAdd the dry ingredients to the wet ingredients and stir until just combined. The dough should be on the dry side. If it’s too dry to form into balls add water, ½ teaspoon at a time, until the dough is just cohesive. If it’s too dry and crumbly to work with, add water, 1 teaspoon at a time, until it comes together.\n\nScoop dough using a 2-inch cookie scoop and roll into balls with your hands. Flatten slightly and place on the baking sheet. Use a fork to make cross-hatch marks. Sprinkle with flaky sea salt, if desired.\n\nBake for 12 to 13 minutes. Remove from the oven and let sit on the baking sheet for 10 minutes before transferring to a wire rack to cool completely."
1,"[2 cups diced strawberries, ½ cup frozen raspberries, thawed, with their juices, Pinches of cane sugar]","Make the macerated berries: In a medium bowl, combine the strawberries, raspberries, and a few pinches of sugar. Set aside for 10 minutes for the berries to soften. Stir before serving.\n\nMake the French toast: In a large bowl, whisk together the eggs, milk, cinnamon, cardamom, and salt. Dip each slice of bread into the mixture and set the soaked bread aside on a large tray or plate.\n\nHeat a non-stick skillet to medium heat and brush with coconut oil. Add the bread slices and cook until golden brown, about 2 minutes per side. Reduce the heat to low as needed to cook thoroughly without burning. Serve with maple syrup and the macerated berries."
2,"[½ cup warm water, 110°, 1 (¼-ounce) package active dry yeast, 1 teaspoon cane sugar, ⅓ cup melted coconut oil, plus more for brushing, ½ cup almond milk, at room temperature, ⅓ cup cane sugar, 1 teaspoon sea salt, 2¾ cups all-purpose flour, plus more for kneading, ½ cup brown sugar, 1½ tablespoons cinnamon]","Grease an 8x11 or 9x13-inch baking dish.\n\nIn a small bowl, stir together the water, yeast, and sugar. Set aside for 5 minutes, or until the yeast is foamy.\n\nMake the dough: In a medium bowl, combine the coconut oil, almond milk, sugar, and salt. Stir in the yeast mixture. Place the flour in a large bowl, then add the yeast mixture and stir until combined. The mixture will be sticky. Use your hands to roughly knead the mixture, then turn it out onto a floured surface. Knead 3 to 4 minutes until smooth, sprinkle with more flour if needed, and form into a ball.\n\nBrush a large bowl with coconut oil and place the dough inside. Cover with plastic wrap and set aside in a warm place to rise until doubled in size, about 1 hour.\n\nMake the filling: In a small bowl, mix the brown sugar and cinnamon.\n\nPunch down dough and roll out on a floured surface into a 20x14-inch rectangle. Brush with 2 tablespoons melted coconut oil and sprinkle with the cinnamon sugar to within ½ inch of the edges.\n\nStarting at one of the short 14-inch ends, roll tightly into a log, then slice into 12 rolls. Place the rolls into the baking dish cut-side up, cover, and let rise for 1 hour.\n\nPreheat the oven to 350°F.\n\nMake the glaze: In a medium bowl, whisk together the powdered sugar, 3 tablespoons almond milk, and vanilla until smooth. If it’s too thick, add more almond milk.\n\nBake the rolls 25 to 30 minutes or until lightly golden on top. Remove and allow to cool for 10 minutes, then drizzle the glaze on top and serve."


### Check the shape and info

In [242]:
len(set_json)

1036

In [243]:
set_json[:1]

[{'Recipe': ['1 tablespoon ground flaxseed',
   '3 tablespoons water',
   '1½ cups all-purpose flour, spooned and leveled',
   '½ teaspoon baking soda',
   '¼ teaspoon baking powder',
   '½ teaspoon sea salt',
   '½ cup coconut oil, soft but not melted'],
  'INSTRUCTIONS': 'Preheat the oven to 350°F and line a baking sheet with parchment paper.\n\nIn a small bowl, combine the flaxseed and water and set aside to thicken for 5 minutes.\n\nIn a medium bowl, combine the (spooned and leveled) flour, baking soda, baking powder, and salt.\n\nIn a large bowl, combine the coconut oil, cane sugar, brown sugar, almond butter, and vanilla. Stir in the flaxseed mixture and mix well.\n\nAdd the dry ingredients to the wet ingredients and stir until just combined. The dough should be on the dry side. If it’s too dry to form into balls add water, ½ teaspoon at a time, until the dough is just cohesive. If it’s too dry and crumbly to work with, add water, 1 teaspoon at a time, until it comes together.\n\

In [244]:
df.shape

(1036, 2)

<b><br> We have 1035 rows and 2 columns. 
<br>`Recipe` and `INSTRUCTIONS` are our 2 columns.
<br> Also our present columns will be our labels for each sentence. 
<br> Later we will split the data for sentences: 
<br>each row will be a sentence and we will have a label 1- recipe and 0 - instructions.</b>

In [245]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Recipe        1036 non-null   object
 1   INSTRUCTIONS  1036 non-null   object
dtypes: object(2)
memory usage: 16.3+ KB


<b>Ok, good. <br>Now let's transform our first column `Recipe` :</b>

In [246]:
def from_list_to_str(series):
    return ' '.join([words for words in series])


recipe_col = df["Recipe"]
recipe = pd.DataFrame(recipe_col.apply(from_list_to_str).to_numpy(), columns=['sentence'])
recipe.head()

,sentence
0,"1 tablespoon ground flaxseed 3 tablespoons water 1½ cups all-purpose flour, spooned and leveled ½ teaspoon baking soda ¼ teaspoon baking powder ½ teaspoon sea salt ½ cup coconut oil, soft but not melted"
1,"2 cups diced strawberries ½ cup frozen raspberries, thawed, with their juices Pinches of cane sugar"
2,"½ cup warm water, 110° 1 (¼-ounce) package active dry yeast 1 teaspoon cane sugar ⅓ cup melted coconut oil, plus more for brushing ½ cup almond milk, at room temperature ⅓ cup cane sugar 1 teaspoon sea salt 2¾ cups all-purpose flour, plus more for kneading ½ cup brown sugar 1½ tablespoons cinnamon"
3,"2 cups cubed butternut squash 2 cups halved Brussels sprouts 2 cups cauliflower florets 2 carrots, cut into 1-inch pieces 1 to 2 turnips, cut into 1-inch pieces"
4,"1¾ cups warm water, 105° to 115°F 1 (¼-ounce) package active dry yeast, (2¼ teaspoons) 1 tablespoon cane sugar 3½ cups all-purpose flour, plus more for kneading 1½ cups whole wheat flour"


<b>We have to add a column `label`:</b>

In [247]:
recipe['label'] = np.ones(len(recipe),int)
recipe.head()

,sentence,label
0,"1 tablespoon ground flaxseed 3 tablespoons water 1½ cups all-purpose flour, spooned and leveled ½ teaspoon baking soda ¼ teaspoon baking powder ½ teaspoon sea salt ½ cup coconut oil, soft but not melted",1
1,"2 cups diced strawberries ½ cup frozen raspberries, thawed, with their juices Pinches of cane sugar",1
2,"½ cup warm water, 110° 1 (¼-ounce) package active dry yeast 1 teaspoon cane sugar ⅓ cup melted coconut oil, plus more for brushing ½ cup almond milk, at room temperature ⅓ cup cane sugar 1 teaspoon sea salt 2¾ cups all-purpose flour, plus more for kneading ½ cup brown sugar 1½ tablespoons cinnamon",1
3,"2 cups cubed butternut squash 2 cups halved Brussels sprouts 2 cups cauliflower florets 2 carrots, cut into 1-inch pieces 1 to 2 turnips, cut into 1-inch pieces",1
4,"1¾ cups warm water, 105° to 115°F 1 (¼-ounce) package active dry yeast, (2¼ teaspoons) 1 tablespoon cane sugar 3½ cups all-purpose flour, plus more for kneading 1½ cups whole wheat flour",1


In [248]:
recipe.to_numpy().shape

(1036, 2)

<b>Great! It's time to prepare the `INSTRUCTIONS` column</b>

In [249]:
instr_col = df["INSTRUCTIONS"].str.split('\n\n').to_numpy()
instr = np.concatenate(instr_col).reshape(-1,1)
instr = pd.DataFrame(instr, columns=['sentence'])
instr.head()

,sentence
0,Preheat the oven to 350°F and line a baking sheet with parchment paper.
1,"In a small bowl, combine the flaxseed and water and set aside to thicken for 5 minutes."
2,"In a medium bowl, combine the (spooned and leveled) flour, baking soda, baking powder, and salt."
3,"In a large bowl, combine the coconut oil, cane sugar, brown sugar, almond butter, and vanilla. Stir in the flaxseed mixture and mix well."
4,"Add the dry ingredients to the wet ingredients and stir until just combined. The dough should be on the dry side. If it’s too dry to form into balls add water, ½ teaspoon at a time, until the dough is just cohesive. If it’s too dry and crumbly to work with, add water, 1 teaspoon at a time, until it comes together."


<b>Adding column `label` with value 0</b>

In [250]:
instr['label'] = np.zeros(len(instr),int)
instr

,sentence,label
0,Preheat the oven to 350°F and line a baking sheet with parchment paper.,0
1,"In a small bowl, combine the flaxseed and water and set aside to thicken for 5 minutes.",0
2,"In a medium bowl, combine the (spooned and leveled) flour, baking soda, baking powder, and salt.",0
3,"In a large bowl, combine the coconut oil, cane sugar, brown sugar, almond butter, and vanilla. Stir in the flaxseed mixture and mix well.",0
4,"Add the dry ingredients to the wet ingredients and stir until just combined. The dough should be on the dry side. If it’s too dry to form into balls add water, ½ teaspoon at a time, until the dough is just cohesive. If it’s too dry and crumbly to work with, add water, 1 teaspoon at a time, until it comes together.",0
...,...,...
4172,"For curly ""spaghetti"" noodles using a countertop spiralizer: Clamp the spiralizer to your countertop. Trim off the tip of the zucchini and secure it between the blade and the teeth of the spiralizer. Turn the handle to make the noodles.",0
4173,"For straight ""angel hair' noodles using a julienne peeler: Simply hold the zucchini with one hand and pull the julienne peeler across the zucchini to make strips.",0
4174,"For ""fettucine"" noodles using a mandoline and a knife: Slice thin zucchini planks with the mandoline, and then cut those planks into fettuccine-sized strips.",0
4175,"For ""pappardelle"" noodles using a regular vegetable peeler: Use the peeler to simply peel thin strips of zucchini.",0


In [251]:
instr.to_numpy().shape

(4177, 2)

In [252]:
data = pd.DataFrame(np.concatenate((instr.to_numpy(), recipe.to_numpy())), columns=['paragraph','label'])
data.head()

,paragraph,label
0,Preheat the oven to 350°F and line a baking sheet with parchment paper.,0
1,"In a small bowl, combine the flaxseed and water and set aside to thicken for 5 minutes.",0
2,"In a medium bowl, combine the (spooned and leveled) flour, baking soda, baking powder, and salt.",0
3,"In a large bowl, combine the coconut oil, cane sugar, brown sugar, almond butter, and vanilla. Stir in the flaxseed mixture and mix well.",0
4,"Add the dry ingredients to the wet ingredients and stir until just combined. The dough should be on the dry side. If it’s too dry to form into balls add water, ½ teaspoon at a time, until the dough is just cohesive. If it’s too dry and crumbly to work with, add water, 1 teaspoon at a time, until it comes together.",0


In [253]:
data.shape

(5213, 2)

In [254]:
data.label[data.label == 1].count()

1036

In [255]:
data.label[data.label == 0].count()

4177

In [256]:
data.isna().sum()

paragraph    0
label        0
dtype: int64

##### no null - good, check the proportion

In [257]:
round(data['label'].value_counts()/len(data)*100,2)

0    80.13
1    19.87
Name: label, dtype: float64

##### drop duplicated and empty rows

In [258]:
data[data.paragraph == '']

,paragraph,label
1619,,0
2463,,0
2588,,0
2810,,0
3478,,0
3793,,0
3897,,0
4024,,0
4084,,0
4136,,0


In [259]:
data = data[data.paragraph != '']
data.shape

(5193, 2)

In [260]:
data = data.drop_duplicates().reset_index(drop=True)
data.tail(50)

,paragraph,label
4799,"3-4 assorted bell peppers ½ cucumber, chopped 1½ cups chickpeas, cooked, or canned and drained",1
4800,"1 tablespoon butter or vegan butter 1 tablespoon extra-virgin olive oil ½ red onion, chopped 1 garlic clove, minced 1 tablespoon lemon thyme leaves or regular thyme",1
4801,"1 pint cherry tomatoes (from your local farmers market, Whole Foods, Trader Joe's, your friend’s garden, etc.) 6-8 ounces brown rice pasta",1
4802,1 recipe peach salsa Corn tortillas,1
4803,"2 firm, yet ripe, peaches 2 teaspoons extra-virgin olive oil 1 teaspoon balsamic vinegar 2 cups baby arugula",1
4804,"1 small red onion, sliced thin Half loaf day-old ciabatta bread ½ garlic clove 1½ cups cherry and/or pear tomatoes, sliced in half ½ cup ricotta salata cheese, chopped into ½-inch cubes",1
4805,"1 poblano pepper 1 ear of corn, kernels sliced off the cob (about ½ cup) ¼ cup sliced cherry tomatoes ½ lime, juiced (1-2 tablespoons) ¼ cup sliced scallions, white and green parts, (about 2-3)",1
4806,"1½ cup almonds, blanched, skins removed* 1½ cups corn kernels, blanched (fresh or frozen) 1 cup water",1
4807,"2 tablespoons extra-virgin olive oil, plus more for drizzling 2 tablespoons lemon juice, plus more for squeezing 1 tablespoon champagne vinegar",1
4808,"2 medium potatoes, chopped into ½-inch pieces 1 garlic clove, minced ½ cup green beans, sliced into 1-inch pieces ½ cup uncooked Israeli couscous, or use quinoa if gluten free ½ cup cherry tomatoes, halved or quartered ½ cup corn (about 1 ear), grilled and sliced off the cob, or raw",1


##### check the proportion again

In [261]:
round(data['label'].value_counts()/len(data)*100,2)

0    79.71
1    20.29
Name: label, dtype: float64

##### Chenged not much, but +6.2%  of sentences in  `ingredients` and -6.2% in `recipe` after droping duplacates
##### We have a little bit imballanced data set
##### Replace `.` to token `zzdot` and split on it

In [262]:
DIGIT_RX = "(\d+([\.|,]\d+)?[\w]*[\s|-])|[^A-Za-z\,()\.'\-: ]{1,7}"
SYMBOL_RX ="[/(/)\-/*/,]|[^ -~]"
DOT_RX = "\.|\:"

def replace_numbers_str(series):
        """
        replace numbers expression (11 ; 11,00;  1111.99; 23-th 25-,1/2,¼) with tag ' zNUM ',
        . and : replace with ' zDOT '
        :param series: pandas series of text strings
        :return: series with replaced numbers
        """
        new_series1 = re.sub(DIGIT_RX," zNUM ",series)
        new_series2 = re.sub(SYMBOL_RX,' ',new_series1)
        new_series = re.sub(DOT_RX," zDOT",new_series2)
        return new_series

data['paragraph'] = data.paragraph.apply(replace_numbers_str)
data.tail(15)

,paragraph,label
4834,zNUM tablespoons extra virgin olive oil zNUM small leeks light green zNUM white parts coarsely chopped zNUM shallot coarsely chopped zNUM garlic cloves coarsely chopped Splash white wine or water zNUM bag frozen green garbanzo beans about zNUM cups zNUM cups vegetable broth,1
4835,zNUM zNUM ounces lump crabmeat zNUM small shallot chopped about zNUM cup zNUM garlic clove minced zNUM tablespoon grated fresh ginger,1
4836,zNUM cup arborio rice zNUM cup white wine zNUM clove garlic minced zNUM large onion chopped zNUM cups veggie stock zNUM tablespoons butter I use non dairy butter zNUM tablespoon olive oil,1
4837,mini ciabatta rolls slightly trim off the zNUM roll zNUM top or another bread of your choice fresh mozzarella slices zNUM big slice per mini sandwich a few sundried tomatoes chopped zNUM packed cups basil,1
4838,zNUM cup raspberries I used frozen lightly mashed zNUM tablespoons butter I use vegan Earth Balance butter zNUM ounces bittersweet chocolate zNUM cup almond milk zNUM cup oat flour zNUM teaspoon baking powder zNUM teaspoon salt zNUM cup maple syrup,1
4839,zNUM cups ground raw cauliflower florets about zNUM small head zNUM cup almond flour zNUM eggs or you could try flax eggs if vegan Sea salt and fresh black pepper zNUM teaspoon onion powder optional,1
4840,Florets from zNUM small head of cauliflower yielding zNUM cups ground zNUM riced zNUM cauliflower zNUM eggs not including the eggs I used as a topping zNUM cup almond flour or more if needed Sea salt and fresh black pepper zNUM teaspoon onion powder optional zNUM teaspoon garlic powder or a bit of minced garlic optional zNUM zNUM tablespoons nutritional yeast or parmesan cheese optional,1
4841,zNUM cup flat leaf parsley zNUM cup basil zNUM cup extra virgin olive oil zNUM tablespoon champagne vinegar Juice of zNUM small lemon about zNUM tablespoons,1
4842,zNUM cup almond flour zNUM cup oat flour zNUM cup rolled oats zNUM cup chocolate chips zNUM cup walnuts or pecans chopped,1
4843,zNUM sweet potato baked peeled and mashed zNUM block of extra firm tofu from a zNUM oz package zNUM tablespoon tahini zNUM teaspoons maple syrup zNUM garlic clove minced zNUM teaspoon minced rosemary zNUM teaspoon paprika zNUM cup panko to roll the patties in before baking Sea salt and fresh black pepper Slider or regular hamburger buns,1


# Uzing numpy + pandas in one step all notebook

In [269]:
def from_list_to_str(series):
    return ' '.join([words for words in series])


# load data
filename='recipes.pkl'
df = pd.DataFrame(pd.read_pickle(filename))



recipe_col = df["Recipe"].apply(from_list_to_str).to_numpy()
# recipe = pd.DataFrame(recipe_col, columns=['sentence'])

# #transform recioe to array and give a label 1
recipe = recipe_col.reshape(-1,1)
recipe = np.hstack((recipe,np.ones(len(recipe),int).reshape(-1,1)))
print(recipe.shape)

# #transform instructions to array and give a label 0
instr_col = df["INSTRUCTIONS"].str.split('\n\n').to_numpy()
instr = np.concatenate(instr_col).reshape(-1,1)
instr = np.hstack((instr,np.zeros(len(instr),int).reshape(-1,1)))
print(instr.shape)

# #forming a full data array with labels
data = np.concatenate((instr, recipe))
print('shape all of paragraphs data matrix',data.shape)
pd.DataFrame(data).tail(50)

(1036, 2)
(4177, 2)
shape all of paragraphs data matrix (5213, 2)


,0,1
5163,"2 tablespoons walnut oil 1 tablespoon extra-virgin olive oil Juice and zest of 1 lemon 1 garlic clove, minced",1
5164,"6-8 slices whole grain bread 1 garlic clove (use half to rub bread with, mince other half for ricotta mixture) 1 tomato ½ cup fresh, high quality ricotta cheese 1-2 tablespoons lemon juice",1
5165,"Strawberries (fresh or frozen) Maple syrup 1 basil leaf, chopped 2 eggs",1
5166,"1 cup raw sunflower seeds, soaked for at least 6 hours 2 tablespoons tahini 2 tablespoons lemon juice 1 tablespoon white miso paste 1 tablespoon minced ginger 1 tablespoon minced garlic 1 teaspoon rice vinegar 1 teaspoon brown rice syrup, honey, or agave",1
5167,"½ pound pasta, homemade or store bought - whole wheat, or whatever you like ¼ cup, plus 1 tablespoon olive oil. 1 shallot, finely chopped 1 garlic clove, minced splash of white wine (1-2 tablespoons - optional) 1 cup chopped asparagus, thin spears, tender parts only",1
5168,"4-5 medium to large tomatillos 1 jalapeño ½ onion, chopped 1 garlic clove, minced ½ cup water A few sprigs cilantro, chopped Splash extra-virgin olive oil for the pan Sea salt 1 poblano pepper",1
5169,"2 cups chopped Swiss chard ½ garlic clove, minced 1 lime, for squeezing",1
5170,"1 fennel bulb, white part only 3 squeezes lemon Splash honey, agave, or sugar A few pinches chopped dill",1
5171,"2 cups asparagus, tender parts only, cut into 1-inch pieces 2 tablespoons sesame seeds 3 tablespoons sliced almonds A few handfuls arugula or other salad greens (about 2 cups) Feta or other cheese (optional)",1
5172,"2 tablespoons extra-virgin olive oil 2 small leeks, light green & white parts, coarsely chopped 1 shallot, coarsely chopped 2 garlic cloves, coarsely chopped Splash white wine or water 1 bag frozen green garbanzo beans (about 2 cups) 1½ cups vegetable broth",1


In [270]:
# # remove duplicates
unique = np.unique(data.astype(str), axis=0)
print('shape without duplicates',unique.shape)
# #remove empty string rows(from table)
unique = np.delete(unique.astype(str),np.where(unique == ''),axis=0)
print('shape without empty string rows',unique.shape)


                                                                                                                                                                                                                                                                                                                                0  \
0                                                                                                                                                                                                                                                         Preheat the oven to 350°F and line a baking sheet with parchment paper.   
1                                                                                                                                                                                                                                         In a small bowl, combine the flaxseed and water and set aside to thicken for 5 minutes.   
2                        

In [271]:
#constants regex
DIGIT_RX = "(\d+([\.|,]\d+)?[\w]*[\s|-])|[^A-Za-z\,()\.'\-: ]{1,7}"
SIMBOL_RX ="[/(/)\-/*/,]|[^ -~]"
DOT_RX = "\.{1,4}|\:" 


def replace_numbers_str(series):
        """
        replace numbers expression (11 ; 11,00;  1111.99; 23-th 25-,1/2,¼) with tag ' zNUM ',
        . and : replace with ' zDOT '
        :param series: pandas series of text strings
        :return: series with replaced numbers
        """
        new_series1 = re.sub(DIGIT_RX," zNUM ",series)
        new_series2 = re.sub(SIMBOL_RX,' ',new_series1)
        new_series = re.sub(DOT_RX," zDOT ",new_series2)
        return new_series
    
    
def lemmatiz(series):
    """Transform all words to lemma, add tag  -PRON- 
    paramLseries:str
    return: series:str"""
    new_series =  ' '.join([word.lemma_ for word in nlp(series)])
    return new_series

def have_pron(series):
    """Give the answer is there a pron in the paragraph
    param:series:str
    return: int 1 or 0"""
    answer = 0
    if series.__contains__('-PRON-'):
        answer = 1
    return answer
    
    
def remove_punctuation(series):
    """ Remove punctuation from each word, make every word to lower case
    params: series of strings
    return transformed series"""
    table = str.maketrans('', '', string.punctuation)
    tokens_punct = series.translate(table).lower()
    return tokens_punct


def remove_stop_words(series):
    """Remove stopwords in the series:str and makes words lower case
    param:series:str
    return: new_series:str
        """
    new_series =  ' '.join([word for word in series.split() if word not in stop_words])
    return new_series


def count_paragraph_sentences(series):
    """Count number of sentences in the paragraph
        :param series: pandas series of text strings
        :return: int count of sentences in the string (if no dots -> count = 1)"""
    sent_count = series.count(' zdot ')
    if sent_count == 0:
        sent_count = 1
    return sent_count


def num_count(series):
    """Count number of sentences in the paragraph
        :param series: pandas series of text strings
        :return: int count of sentences in the string"""
    return series.count('znum') 


def count_words(series):
    """ Count words in each string (paragraph)
    without dots and numbers
    params: series of strings
    returns: new series:int"""
        
    clean_sent_len = len([word for word in series.split() 
                            if word not in['znum','zdot']])
    return clean_sent_len



In [272]:
#transform to pandas -> easy to clean    
data = pd.DataFrame(unique, columns=['paragraph', 'label'])
data['paragraph'] = data.paragraph.apply(replace_numbers_str)

#this one takes an eternity (lemmatiz)
data['lemmatiz'] = data.paragraph.apply(lemmatiz)
data['contains_pron'] = data.lemmatiz.apply(have_pron)
data['tokens_punct'] = data.lemmatiz.apply(remove_punctuation)

data['remove_stop_words'] = data.tokens_punct.apply(remove_stop_words)

data['sent_count'] = data.remove_stop_words.apply(count_paragraph_sentences)
data['num_count'] = data.remove_stop_words.apply(num_count)
data['clean_paragraph_len'] = data.remove_stop_words.apply(count_words)
data['not_clean_paragraph_len'] = data.paragraph.apply(count_words)
# data.drop('paragraph', axis=1, inplace=True)

data.head()

,paragraph,label,lemmatiz,contains_pron,tokens_punct,remove_stop_words,sent_count,num_count,clean_paragraph_len,not_clean_paragraph_len
0,Alternatively omit the sparkling water shake everything in a cocktail shaker with ice and serve in a rocks glass zDOT,0,alternatively omit the sparkling water shake everything in a cocktail shaker with ice and serve in a rock glass zdot,0,alternatively omit the sparkling water shake everything in a cocktail shaker with ice and serve in a rock glass zdot,alternatively omit sparkling water shake cocktail shaker ice serve rock glass zdot,1,0,11,20
1,Optional zDOT if you're making okra drizzle with olive oil salt zNUM pepper and roast at the same time for about zNUM minutes or until they start to brown zDOT,0,optional zdot if -PRON- be make okra drizzle with olive oil salt zNUM pepper and roast at the same time for about znum minute or until -PRON- start to brown zdot,1,optional zdot if pron be make okra drizzle with olive oil salt znum pepper and roast at the same time for about znum minute or until pron start to brown zdot,optional zdot pron okra drizzle olive oil salt znum pepper roast time znum minute pron start brown zdot,1,2,14,30
2,also good on pasta in salads etc zDOT,0,also good on pasta in salad etc zdot,0,also good on pasta in salad etc zdot,good pasta salad etc zdot,1,0,4,8
3,optional I served mine on top of a few handfuls of arugula zDOT,0,optional -PRON- serve -PRON- on top of a few handful of arugula zDOT,1,optional pron serve pron on top of a few handful of arugula zdot,optional pron serve pron handful arugula zdot,1,0,6,13
4,optional step Mix your yogurt sauce together and pop it in the fridge zDOT,0,optional step mix -PRON- yogurt sauce together and pop -PRON- in the fridge zDOT,1,optional step mix pron yogurt sauce together and pop pron in the fridge zdot,optional step mix pron yogurt sauce pop pron fridge zdot,1,0,9,14


In [273]:
data[5000:5030] #5020

,paragraph,label,lemmatiz,contains_pron,tokens_punct,remove_stop_words,sent_count,num_count,clean_paragraph_len,not_clean_paragraph_len


In [274]:

data[4171:4180]

,paragraph,label,lemmatiz,contains_pron,tokens_punct,remove_stop_words,sent_count,num_count,clean_paragraph_len,not_clean_paragraph_len
4171,Store extra pumpkin spread in the fridge for zNUM to zNUM days zDOT Top it on tacos serve with quesadillas or use it as a dip for chips zDOT,0,store extra pumpkin spread in the fridge for znum to zNUM day zdot top -PRON- on tacos serve with quesadillas or use -PRON- as a dip for chip zDOT,1,store extra pumpkin spread in the fridge for znum to znum day zdot top pron on tacos serve with quesadillas or use pron as a dip for chip zdot,store extra pumpkin spread fridge znum znum day zdot pron tacos serve quesadillas use pron dip chip zdot,1,2,14,29
4172,Store extra romesco sauce in the fridge for up to zNUM days zDOT Slather it on sandwiches or use as a veggie dip zDOT,0,store extra romesco sauce in the fridge for up to zNUM day zDOT slather -PRON- on sandwich or use as a veggie dip zDOT,1,store extra romesco sauce in the fridge for up to znum day zdot slather pron on sandwich or use as a veggie dip zdot,store extra romesco sauce fridge znum day zdot slather pron sandwich use veggie dip zdot,1,1,12,24
4173,Store extra sauce in the fridge it's also great as a sandwich spread or a veggie dip zDOT,0,store extra sauce in the fridge -PRON- be also great as a sandwich spread or a veggie dip zDOT,1,store extra sauce in the fridge pron be also great as a sandwich spread or a veggie dip zdot,store extra sauce fridge pron great sandwich spread veggie dip zdot,1,0,10,18
4174,Store extra spread at room temperature in an airtight jar zDOT,0,store extra spread at room temperature in an airtight jar zDOT,0,store extra spread at room temperature in an airtight jar zdot,store extra spread room temperature airtight jar zdot,1,0,7,11
4175,Store extra sun cheese in the fridge it'll last at least zNUM zNUM more days zDOT If it starts to separate just give it a stir zDOT,0,store extra sun cheese in the fridge -PRON- will last at least znum zNUM more day zdot if -PRON- start to separate just give -PRON- a stir zDOT,1,store extra sun cheese in the fridge pron will last at least znum znum more day zdot if pron start to separate just give pron a stir zdot,store extra sun cheese fridge pron znum znum day zdot pron start separate pron stir zdot,1,2,12,27
4176,Store frosted cake in the fridge zDOT,0,store frost cake in the fridge zDOT,0,store frost cake in the fridge zdot,store frost cake fridge zdot,1,0,4,7
4177,Store in an air tight container in the refrigerator zDOT,0,store in an air tight container in the refrigerator zDOT,0,store in an air tight container in the refrigerator zdot,store air tight container refrigerator zdot,1,0,5,10
4178,Store in the fridge for up to zNUM days freeze after that zDOT,0,store in the fridge for up to znum day freeze after that zDOT,0,store in the fridge for up to znum day freeze after that zdot,store fridge znum day freeze zdot,1,1,4,13
4179,Store in the refrigerator for up to two weeks zDOT,0,store in the refrigerator for up to two week zDOT,0,store in the refrigerator for up to two week zdot,store refrigerator week zdot,1,0,3,10


In [275]:
df = data[['remove_stop_words','sent_count','num_count','clean_paragraph_len','contains_pron','label']]

df

,remove_stop_words,sent_count,num_count,clean_paragraph_len,contains_pron,label
0,alternatively omit sparkling water shake cocktail shaker ice serve rock glass zdot,1,0,11,0,0
1,optional zdot pron okra drizzle olive oil salt znum pepper roast time znum minute pron start brown zdot,1,2,14,1,0
2,good pasta salad etc zdot,1,0,4,0,0
3,optional pron serve pron handful arugula zdot,1,0,6,1,0
4,optional step mix pron yogurt sauce pop pron fridge zdot,1,0,9,1,0
...,...,...,...,...,...,...
4844,znum cup toast walnut pine nut znum small clove garlic znum cup coarsely chop carrot preferably organic big handful basil leave znum cup,1,4,19,0,1
4845,znum cup milk greek yogurt,1,1,4,0,1
4846,znum cup roll oats znum cup chop pecans znum cup flour kind zdot spelt oat wheat purpose gf blend znum cup brown sugar,1,4,18,0,1
4847,znum cup good quality coffee bean buy pron pre grind zdot znum cup filter water almond breeze almondmilk original znum cup latte,1,3,18,1,1


In [276]:
def save_data_to_pkl(json_file, file):
    """The function receive  file
     dumps into the pkl file
    :return full path of saved file"""
    pickle.dump(json_file, open(file,'wb'), pickle.HIGHEST_PROTOCOL)
    return file


path_to_data = save_data_to_pkl(df, 'data_clean.pkl' )
print(path_to_data)




data_clean.pkl
